In [3]:
import os 
import time
import warnings
import numpy as np
import pandas as pd 
from datetime import datetime as dt
from datetime import timedelta 
warnings.filterwarnings('ignore')
#Document Loader
from langchain_community.document_loaders import PyPDFLoader   

#Splitting data to manageable chunks and vectorizing; in memory-store
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

#Langchain Imports
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document

#HuggingFace Embeddings
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

#Retrieval QA 
from langchain.chains import RetrievalQA 
from langchain.schema import SystemMessage, HumanMessage

from dotenv import load_dotenv
import faiss


load_dotenv()

False

In [5]:
model_name = 'Alibaba-NLP/gte-large-en-v1.5'

In [6]:
embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs={"trust_remote_code": True}
)

In [7]:
def load_documents(file_path):
    """
    Load PDF documents from the specified file path.
    """
    loader = PyPDFLoader(file_path)
    return loader.load()

In [9]:
document = load_documents('document/documents.pdf')

Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 124 0 (offset 0)
Ignoring wrong pointing object 138 0 (offset 0)
Ignoring wrong pointing object 194 0 (offset 0)
Ignoring wrong pointing object 201 0 (offset 0)
Ignoring wrong pointing object 202 0 (offset 0)


In [10]:
#Split the text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    #Set a really small chunk size, so that we can test the system
    chunk_size = 1000,
    chunk_overlap = 100,
)

In [11]:
docs= text_splitter.split_documents(document)

In [12]:
texts = [doc.page_content for doc in docs]

In [13]:
#Create a Vector Store
vectorstore = FAISS.from_texts(
    texts = texts,
    embedding= embedding)

In [14]:
#Create a retriever 
retriever = vectorstore.as_retriever()

In [15]:
Prompt_Template = """
You are a tax Expert, your job is to answer questions about tax laws and regulations.
You will be provided with a context from a tax document, and you will answer the question based on that context."""

In [16]:
PROMPT = PromptTemplate(
    template=Prompt_Template,
    input_variables=["context", "question"],
)